In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

2025-04-07 00:14:13.464590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743981253.476604   24445 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743981253.480208   24445 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743981253.488951   24445 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743981253.488963   24445 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743981253.488964   24445 computation_placer.cc:177] computation placer alr

In [4]:
print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [26]:
goes_test = pd.read_csv('data/dataset/test/goes/goes-00000-20000603_to_20000802.csv')
goes_test['Timestamp'] = pd.to_datetime(goes_test['Timestamp'])

In [33]:
goes_df = goes_test.set_index('Timestamp').resample('0.25H').interpolate('time')
goes_df

/tmp/ipykernel_24445/1486537641.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  goes_df = goes_test.set_index('Timestamp').resample('0.25H').interpolate('time')


,quad_diode,xrsa_flux,xrsa_flux_observed,xrsa_flux_electrons,xrsb_flux,xrsb_flux_observed,xrsb_flux_electrons,xrsa_flag,xrsb_flag,xrsa_num,...,xrsb2_flag,xrsa1_num,xrsa2_num,xrsb1_num,xrsb2_num,xrsa1_flag_excluded,xrsa2_flag_excluded,xrsb1_flag_excluded,xrsb2_flag_excluded,yaw_flip_flag
Timestamp,,,,,,,,,,,,,,,,,,,,,
2000-06-03 04:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-03 05:00:00,NaN,6.407059e-08,NaN,NaN,2.871429e-06,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-03 05:15:00,NaN,6.720000e-08,NaN,NaN,2.800000e-06,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-03 05:30:00,NaN,5.682353e-08,NaN,NaN,2.685714e-06,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-03 05:45:00,NaN,5.616471e-08,NaN,NaN,2.542857e-06,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-08-02 03:45:00,NaN,1.037647e-08,NaN,NaN,9.842857e-07,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-08-02 04:00:00,NaN,5.550588e-09,NaN,NaN,1.001429e-06,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-08-02 04:15:00,NaN,5.336470e-09,NaN,NaN,9.814286e-07,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(goes_test.to_string())

In [7]:
omni = pd.read_csv('data/dataset/test/omni2/omni2-00000-20000603_to_20000802.csv')
omni["Timestamp"] = pd.to_datetime(omni['Timestamp'])

In [32]:
omni_df = omni.set_index('Timestamp').resample('0.25H').interpolate('time')
omni_df

/tmp/ipykernel_24445/4151579430.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  omni_df = omni.set_index('Timestamp').resample('0.25H').interpolate('time')


,YEAR,DOY,Hour,Bartels_rotation_number,ID_for_IMF_spacecraft,ID_for_SW_Plasma_spacecraft,num_points_IMF_averages,num_points_Plasma_averages,Scalar_B_nT,Vector_B_Magnitude_nT,...,AU_index_nT,pc_index,Lyman_alpha,Proton_flux_>1_Mev,Proton_flux_>2_Mev,Proton_flux_>4_Mev,Proton_flux_>10_Mev,Proton_flux_>30_Mev,Proton_flux_>60_Mev,Flux_FLAG
Timestamp,,,,,,,,,,,,,,,,,,,,,
2000-06-03 05:00:00,2000.0,155.0,5.00,2277.0,71.0,71.0,15.0,15.0,4.6,1.9,...,184.00,1.500,0.008374,0.5000,0.4800,0.4600,0.450,0.3800,0.300,-1.0
2000-06-03 05:15:00,2000.0,155.0,5.25,2277.0,71.0,71.0,15.0,15.0,4.5,2.3,...,175.50,1.525,0.008374,0.4875,0.4675,0.4475,0.435,0.3725,0.295,-1.0
2000-06-03 05:30:00,2000.0,155.0,5.50,2277.0,71.0,71.0,15.0,15.0,4.4,2.7,...,167.00,1.550,0.008374,0.4750,0.4550,0.4350,0.420,0.3650,0.290,-1.0
2000-06-03 05:45:00,2000.0,155.0,5.75,2277.0,71.0,71.0,15.0,15.0,4.3,3.1,...,158.50,1.575,0.008374,0.4625,0.4425,0.4225,0.405,0.3575,0.285,-1.0
2000-06-03 06:00:00,2000.0,155.0,6.00,2277.0,71.0,71.0,15.0,15.0,4.2,3.5,...,150.00,1.600,0.008374,0.4500,0.4300,0.4100,0.390,0.3500,0.280,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-08-02 03:00:00,2000.0,215.0,3.00,2280.0,71.0,71.0,14.0,14.0,6.4,5.7,...,223.00,1.200,0.009013,2.8200,1.2000,0.6700,0.470,0.3700,0.280,-1.0
2000-08-02 03:15:00,2000.0,215.0,3.25,2280.0,71.0,71.0,14.5,14.5,6.3,5.6,...,242.75,1.600,0.009013,2.8775,1.2250,0.6725,0.470,0.3650,0.275,-1.0
2000-08-02 03:30:00,2000.0,215.0,3.50,2280.0,71.0,71.0,15.0,15.0,6.2,5.5,...,262.50,2.000,0.009013,2.9350,1.2500,0.6750,0.470,0.3600,0.270,-1.0


In [34]:
X = pd.concat([goes_df, omni_df],axis=1)
X

,quad_diode,xrsa_flux,xrsa_flux_observed,xrsa_flux_electrons,xrsb_flux,xrsb_flux_observed,xrsb_flux_electrons,xrsa_flag,xrsb_flag,xrsa_num,...,AU_index_nT,pc_index,Lyman_alpha,Proton_flux_>1_Mev,Proton_flux_>2_Mev,Proton_flux_>4_Mev,Proton_flux_>10_Mev,Proton_flux_>30_Mev,Proton_flux_>60_Mev,Flux_FLAG
Timestamp,,,,,,,,,,,,,,,,,,,,,
2000-06-03 04:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-03 05:00:00,NaN,6.407059e-08,NaN,NaN,2.871429e-06,NaN,NaN,0.0,0.0,NaN,...,184.00,1.500,0.008374,0.5000,0.4800,0.4600,0.450,0.3800,0.300,-1.0
2000-06-03 05:15:00,NaN,6.720000e-08,NaN,NaN,2.800000e-06,NaN,NaN,0.0,0.0,NaN,...,175.50,1.525,0.008374,0.4875,0.4675,0.4475,0.435,0.3725,0.295,-1.0
2000-06-03 05:30:00,NaN,5.682353e-08,NaN,NaN,2.685714e-06,NaN,NaN,0.0,0.0,NaN,...,167.00,1.550,0.008374,0.4750,0.4550,0.4350,0.420,0.3650,0.290,-1.0
2000-06-03 05:45:00,NaN,5.616471e-08,NaN,NaN,2.542857e-06,NaN,NaN,0.0,0.0,NaN,...,158.50,1.575,0.008374,0.4625,0.4425,0.4225,0.405,0.3575,0.285,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-08-02 03:45:00,NaN,1.037647e-08,NaN,NaN,9.842857e-07,NaN,NaN,0.0,0.0,NaN,...,282.25,2.400,0.009013,2.9925,1.2750,0.6775,0.470,0.3550,0.265,-1.0
2000-08-02 04:00:00,NaN,5.550588e-09,NaN,NaN,1.001429e-06,NaN,NaN,0.0,0.0,NaN,...,302.00,2.800,0.009013,3.0500,1.3000,0.6800,0.470,0.3500,0.260,-1.0
2000-08-02 04:15:00,NaN,5.336470e-09,NaN,NaN,9.814286e-07,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
mask = (~pd.isnull(X)).astype(int)
valid_mask = (X['xrsa_flag'] == 0.0) & (X['xrsb_flag'] == 0.0).astype(int)
valid_mask = mask.mul(valid_mask.values,axis=0)
valid_mask


,quad_diode,xrsa_flux,xrsa_flux_observed,xrsa_flux_electrons,xrsb_flux,xrsb_flux_observed,xrsb_flux_electrons,xrsa_flag,xrsb_flag,xrsa_num,...,AU_index_nT,pc_index,Lyman_alpha,Proton_flux_>1_Mev,Proton_flux_>2_Mev,Proton_flux_>4_Mev,Proton_flux_>10_Mev,Proton_flux_>30_Mev,Proton_flux_>60_Mev,Flux_FLAG
Timestamp,,,,,,,,,,,,,,,,,,,,,
2000-06-03 04:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000-06-03 05:00:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-06-03 05:15:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-06-03 05:30:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-06-03 05:45:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-08-02 03:45:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-08-02 04:00:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-08-02 04:15:00,0,1,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


,quad_diode,xrsa_flux,xrsa_flux_observed,xrsa_flux_electrons,xrsb_flux,xrsb_flux_observed,xrsb_flux_electrons,xrsa_flag,xrsb_flag,xrsa_num,...,AU_index_nT,pc_index,Lyman_alpha,Proton_flux_>1_Mev,Proton_flux_>2_Mev,Proton_flux_>4_Mev,Proton_flux_>10_Mev,Proton_flux_>30_Mev,Proton_flux_>60_Mev,Flux_FLAG
Timestamp,,,,,,,,,,,,,,,,,,,,,
2000-06-03 04:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000-06-03 05:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000-06-03 05:30:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-06-03 06:00:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-06-03 06:30:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-08-02 02:30:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-08-02 03:00:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2000-08-02 03:30:00,0,1,0,0,1,0,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
input = pd.read_csv('input_data/00000_to_02284-initial_states.csv')


In [35]:
sat_dens = pd.read_csv('data/dataset/test/sat_density/gr-of1-05419-20200924_to_20200927.csv')

In [44]:
sat_dens
# plt.scatter(pd.to_datetime(sat_dens['Timestamp']), sat_dens['Orbit Mean Density (kg/m^3)'],s=2)
# plt.xticks(rotation=90)

,Timestamp,Orbit Mean Density (kg/m^3)
0,2020-09-24 20:40:10,6.595480e-14
1,2020-09-24 20:50:10,6.759814e-14
2,2020-09-24 21:00:10,6.803710e-14
3,2020-09-24 21:10:10,6.804441e-14
4,2020-09-24 21:20:10,6.748815e-14
...,...,...
427,2020-09-27 19:50:10,8.233267e-14
428,2020-09-27 20:00:10,8.377393e-14
429,2020-09-27 20:10:10,8.420314e-14
430,2020-09-27 20:20:10,8.493786e-14


In [21]:
input
goes_test

,Timestamp,xrsa_flux,xrsb_flux,xrsa_flag,xrsb_flag
0,2000-06-03 04:51:00,6.637647e-08,0.000003,0.0,0.0
1,2000-06-03 04:52:00,6.604706e-08,0.000003,0.0,0.0
2,2000-06-03 04:53:00,6.703530e-08,0.000003,0.0,0.0
3,2000-06-03 04:54:00,6.588235e-08,0.000003,0.0,0.0
4,2000-06-03 04:55:00,6.456470e-08,0.000003,0.0,0.0
...,...,...,...,...,...
86395,2000-08-02 04:46:00,1.289647e-08,0.000001,0.0,0.0
86396,2000-08-02 04:47:00,1.212235e-08,0.000001,0.0,0.0
86397,2000-08-02 04:48:00,1.200706e-08,0.000001,0.0,0.0
86398,2000-08-02 04:49:00,1.075529e-08,0.000001,0.0,0.0


In [24]:
for column in omni.columns:
    print (column)

YEAR
DOY
Hour
Bartels_rotation_number
ID_for_IMF_spacecraft
ID_for_SW_Plasma_spacecraft
num_points_IMF_averages
num_points_Plasma_averages
Scalar_B_nT
Vector_B_Magnitude_nT
Lat_Angle_of_B_GSE
Long_Angle_of_B_GSE
BX_nT_GSE_GSM
BY_nT_GSE
BZ_nT_GSE
BY_nT_GSM
BZ_nT_GSM
RMS_magnitude_nT
RMS_field_vector_nT
RMS_BX_GSE_nT
RMS_BY_GSE_nT
RMS_BZ_GSE_nT
SW_Plasma_Temperature_K
SW_Proton_Density_N_cm3
SW_Plasma_Speed_km_s
SW_Plasma_flow_long_angle
SW_Plasma_flow_lat_angle
Alpha_Prot_ratio
sigma_T_K
sigma_n_N_cm3
sigma_V_km_s
sigma_phi_V_degrees
sigma_theta_V_degrees
sigma_ratio
Flow_pressure
E_electric_field
Plasma_Beta
Alfen_mach_number
Magnetosonic_Mach_number
Quasy_Invariant
Kp_index
R_Sunspot_No
Dst_index_nT
ap_index_nT
f10.7_index
AE_index_nT
AL_index_nT
AU_index_nT
pc_index
Lyman_alpha
Proton_flux_>1_Mev
Proton_flux_>2_Mev
Proton_flux_>4_Mev
Proton_flux_>10_Mev
Proton_flux_>30_Mev
Proton_flux_>60_Mev
Flux_FLAG
Timestamp


In [ ]:
plt.scatter(omni["Timestamp"],omni['Bartels_rotation_number'],s=2)
plt.xticks(rotation=90)

In [22]:
print(goes_test.shape, omni.shape, input.shape)
print()
print(goes_test.shape[1]+omni.shape[1]+input.shape[1])

(86400, 5) (1440, 58) (2285, 11)

74


In [8]:
X = pd.concat([goes_test, omni, input],axis=1)
X.T.to_numpy

Y = sat_dens['Orbit Mean Density (kg/m^3)'].to_numpy


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


model = Sequential()
model.add(layers.InputLayer(input_shape=(28, 28)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))


model.summary()



/home/ardrit/anaconda3/envs/app_py_env/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1743801594.994311   25221 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 28, 32)         │           928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 28, 32)         │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 28, 32)         │         1,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,040 (11.88 KB)

 Trainable params: 3,040 (11.88 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(loss=SparseCategoricalCrossentropy(), 
              optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [17]:
print(X.dtypes)

Timestamp                     object
xrsa_flux                    float64
xrsb_flux                    float64
xrsa_flag                    float64
xrsb_flag                    float64
                              ...   
Argument of Perigee (deg)    float64
True Anomaly (deg)           float64
Latitude (deg)               float64
Longitude (deg)              float64
Altitude (km)                float64
Length: 74, dtype: object


In [14]:
model.fit(X,Y)

ValueError: Unrecognized data type: x=                 Timestamp     xrsa_flux  xrsb_flux  xrsa_flag  xrsb_flag  \
0      2000-06-03 04:51:00  6.637647e-08   0.000003        0.0        0.0   
1      2000-06-03 04:52:00  6.604706e-08   0.000003        0.0        0.0   
2      2000-06-03 04:53:00  6.703530e-08   0.000003        0.0        0.0   
3      2000-06-03 04:54:00  6.588235e-08   0.000003        0.0        0.0   
4      2000-06-03 04:55:00  6.456470e-08   0.000003        0.0        0.0   
...                    ...           ...        ...        ...        ...   
86395  2000-08-02 04:46:00  1.289647e-08   0.000001        0.0        0.0   
86396  2000-08-02 04:47:00  1.212235e-08   0.000001        0.0        0.0   
86397  2000-08-02 04:48:00  1.200706e-08   0.000001        0.0        0.0   
86398  2000-08-02 04:49:00  1.075529e-08   0.000001        0.0        0.0   
86399  2000-08-02 04:50:00  9.816470e-09   0.000001        0.0        0.0   

         YEAR    DOY  Hour  Bartels_r